# Sentiment Analysis Recommendation System NLP Preprocessing

## Group: U_Data

    Members:
    
    Aroob Almutairi 
    
    Abdulrahman Alosaimi
    
    Duaa Al saad 
    
    Fatimah Alateeq
    
    Ghaliah mohammedhussein

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## Required installation 

In [2]:
# make sure to create a new environment in anaconda navigator with python version 3.6
# ! pip install pandas
# ! pip install numpy
# ! pip install scikit-learn
# ! pip install matplotlib
# ! pip install seaborn
# ! pip install pyarabic
# ! pip install emoji
# ! pip install pystemmer
# ! pip install optuna==2.3.0
# ! pip install transformers==4.2.1
# ! pip install torch

## NLP Preprocessing

In [3]:
import numpy as np
import pandas as pd
import pyarabic.araby as ar

import re , emoji, Stemmer, functools, operator, string
import torch , optuna, gc, random, os

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample

import logging

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)


import matplotlib.pyplot as plt 
import seaborn as sns


/Users/Ghaliah_Maher/opt/anaconda3/envs/DL/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df_NLP= pd.read_excel('NLP_Dataset.xlsx', engine='openpyxl')
df_NLP

,TWEET,LABEL
0,انا معصبه مرا,anger
1,أنا معصبة اتركوني بحالي,anger
2,خلااااااص,anger
3,انا جداً منزعجه,anger
4,هذا انسان كاذب,anger
...,...,...
4036,كئيب ولا يستحق الزيارة,sadness
4037,الغرف كئيبه ومظلمه وتهويتها سيئه,sadness
4038,جو المتحف كئيب نسبيا,sadness
4039,حرام انه عدد الزوار قليل جداً,sympathy


## Droping and Renaming columns

In [5]:
df_NLP.dropna(inplace=True)
df_NLP.rename(columns = {'TWEET':'tweet', 'LABEL':'label'}, inplace = True)
df_NLP

,tweet,label
0,انا معصبه مرا,anger
1,أنا معصبة اتركوني بحالي,anger
2,خلااااااص,anger
3,انا جداً منزعجه,anger
4,هذا انسان كاذب,anger
...,...,...
4036,كئيب ولا يستحق الزيارة,sadness
4037,الغرف كئيبه ومظلمه وتهويتها سيئه,sadness
4038,جو المتحف كئيب نسبيا,sadness
4039,حرام انه عدد الزوار قليل جداً,sympathy


In [6]:
df_NLP.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4021 entries, 0 to 4040
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   4021 non-null   object
 1   label   4021 non-null   object
dtypes: object(2)
memory usage: 94.2+ KB


## Cleanning the Data 

In [7]:
# Removing the punctuations

df_NLP['tweet']=df_NLP['tweet'].str.replace(r'\:', '', regex=True)
df_NLP['tweet']=df_NLP['tweet'].str.replace(r'\.', '', regex=True)
df_NLP['tweet']=df_NLP['tweet'].str.replace(r'\%', '', regex=True)

In [8]:
# Removing digits and english letters

df_NLP['tweet']=df_NLP['tweet'].str.replace(r'\s*[A-Za-z]+\b', '', regex=True)
df_NLP['tweet']=df_NLP['tweet'].str.replace(r'\s*[0-9]+\b', '', regex=True)
df_NLP['tweet']=df_NLP['tweet'].str.replace(r'[^\w\s]', '', regex=True)

In [9]:
df_NLP['tweet']

0                                       انا معصبه مرا
1                             أنا معصبة اتركوني بحالي
2                                           خلااااااص
3                                      انا جدا منزعجه
4                                      هذا انسان كاذب
                            ...                      
4036                           كئيب ولا يستحق الزيارة
4037                 الغرف كئيبه ومظلمه وتهويتها سيئه
4038                            جو المتحف كئيب نسبيا 
4039                     حرام انه عدد الزوار قليل جدا
4040    حرام والله ما في اعلان ولا تعريف بحاجات زي دي
Name: tweet, Length: 4021, dtype: object

In [10]:
# Mix the data
df_NLP_new = df_NLP.sample(frac = 1)
df_NLP_new.reset_index(drop=True)

,tweet,label
0,اف مافي شي راضي يزبط,anger
1,مبسوطه واحساس بالراحه وتنفس مريح,joy
2,متعاطفه مرا مع هذا الشخص,sympathy
3,أنا مكتئبة جدا احتاج عزلة عن الناس,sadness
4,احساس بعدم الراحه قلق وشك نبضات قلب سريعه تنفس...,fear
...,...,...
4016,انا اقوى من الملل,bored
4017,مره يعصب صدق,anger
4018,اشعر بنيران تخرج من عيناي,anger
4019,مصدومة,surprise


In [15]:
train_data=df_NLP_new.iloc[0:3500]
test_data=df_NLP_new.iloc[3501:4020]

In [16]:
train_data['label'].unique()

array(['anger', 'joy', 'sympathy', 'sadness', 'fear', 'bored', 'surprise',
       'none', 'love'], dtype=object)

In [17]:
df_NLP_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4021 entries, 16 to 3512
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   4021 non-null   object
 1   label   4021 non-null   object
dtypes: object(2)
memory usage: 94.2+ KB


## Saving the Results

In [18]:
df_NLP_new.to_csv('clean_NLP_data_set.csv', encoding='utf-8')